In [1]:
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql import Window
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1663742679722_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
acc = sc.accumulator(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
window = Window.partitionBy('User_id').orderBy('Time_Stamp')

def somefunc(value,id):
  global acc

  if   value == '0' : 
    acc = 1
    return('Session_id '+str(acc)+' of '+id)

  elif value > 30:
    acc+=1
    return('Session_id '+str(acc)+' of '+id)
  else:
    return('Session_id '+str(acc)+' of '+id)
        
udfsomefunc = udf(somefunc, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df = spark.read.csv(
    "s3://aws-emr-resources-971996090633-ap-south-1/notebooks/e-DWW3JFF2I6LPKWT6PH45NCHKV/Tech_talk_Data.csv",
    header=True,
    inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df.createOrReplaceTempView("data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+--------------+
|          User_id|  URL|    Time_Stamp|
+-----------------+-----+--------------+
|The Giant Company|page1|10/29/22 11:10|
|       Giant Food|page1|10/29/22 11:10|
|      Stop & Shop|page1|10/29/22 11:10|
+-----------------+-----+--------------+
only showing top 3 rows

In [7]:
data_min = spark.sql(""" 
select User_id, Time_stamp,
case when diff is null then 0
else (unix_timestamp(to_timestamp(Time_Stamp,"dd/mm/yy HH:mm"))- unix_timestamp(diff))/60 end as session_difference
from (select User_id, URL, Time_Stamp, 
lag(to_timestamp(Time_Stamp,"dd/mm/yy HH:mm"),1) over(partition by User_id order by unix_timestamp(to_timestamp(Time_Stamp,"dd/mm/yy HH:mm")) asc) as diff
 from data)""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
data_min.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------+------------------+
|    User_id|    Time_stamp|session_difference|
+-----------+--------------+------------------+
|Stop & Shop| 10/29/22 0:11|               0.0|
|Stop & Shop| 10/29/22 0:41|              30.0|
|Stop & Shop|10/29/22 11:10|             629.0|
|Stop & Shop|10/31/22 11:18|               8.0|
+-----------+--------------+------------------+
only showing top 4 rows

In [15]:
Session_data = data_min.withColumn("Session", udfsomefunc("session_difference","User_id")).drop(col('session_difference'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
Session_data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------+--------------------+
|    User_id|    Time_stamp|             Session|
+-----------+--------------+--------------------+
|Stop & Shop| 10/29/22 0:11|Session_id 0 of S...|
|Stop & Shop| 10/29/22 0:41|Session_id 0 of S...|
|Stop & Shop|10/29/22 11:10|Session_id 1 of S...|
|Stop & Shop|10/31/22 11:18|Session_id 1 of S...|
|     GF/TGC| 10/29/22 0:11|Session_id 0 of G...|
+-----------+--------------+--------------------+
only showing top 5 rows

In [16]:
# # data_min = spark.sql(""" select User_id,Time_Stamp,case when diff is null then 0
# # else datediff(diff,Time_stamp) end as session from (select User_id, URL, Time_Stamp, lag(Time_Stamp,1) over(partition by User_id order by Time_Stamp) as diff from data)"""
# #                     )

# data_min.sort(col('User_id'), col('Time_Stamp')).show(12)

# from pyspark.sql.functions import *

# Session_data = data_min.withColumn(
#     'DiffInSeconds',
#     col("Time_Stamp").cast("long") - col('diff').cast("long"))
# df2.show(truncate=False)







# df.printSchema()

# df.printSchema()

# schema = StructType([
#     StructField("User_id", StringType(), True),
#     StructField("URL", IntegerType(), False),
#     StructField("Time_Stamp", TimestampType(), False)
# ])

# df.collect()

# df.printSchema()

# sc

# acc = sc.accumulator(1)

# window = Window.partitionBy('User_id').orderBy('Time_Stamp')


# def somefunc(value, id):
#     global acc

#     if value is None:
#         acc = 1
#         return ('Session_id ' + str(acc) + ' of ' + id)

#     elif value > 30:
#         acc += 1
#         return ('Session_id ' + str(acc) + ' of ' + id)
#     else:
#         return ('Session_id ' + str(acc) + ' of ' + id)


# udfsomefunc = F.udf(somefunc, StringType())

# from pyspark.sql.functions import *

# df2 = df.withColumn("unix_timestamp", unix_timestamp(F.col("Time_Stamp")))

# df2.show(3)

# df.show()

# df3 = df2.withColumn(
#     'minutes',
#     ((df.unix_timestamp - funcs.lag(df.unix_timestamp, 1).over(window)) /
#      60).cast(IntegerType())).withColumn("Session",
#                                          udfsomefunc("minutes", "SWID"))

# import pyspark.sql.functions as F

# df3 = df2.withColumn("lag", F.lag(F.unix_timestamp, 1).over(window))

